# First Phrase

In [50]:
!pip install rdkit tensorrt

## Get library

In [51]:
# Normal Lib
import os
import pandas as pd
import numpy as np
import math



# keras
from keras import Model
import keras
from keras.layers import Conv3D, Input, MaxPooling3D, BatchNormalization, Dense, Dropout, Flatten,Activation, GlobalAveragePooling3D
import tensorflow as tf
from keras.optimizers import Adam

# Metrics
from scipy.stats import pearsonr # Pearson R best
from keras.metrics import MeanAbsoluteError
from sklearn.metrics import matthews_corrcoef

In [52]:
print(tf.config.list_physical_devices('GPU'))

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [53]:
# !pip install rdkit
from rdkit import Chem

In [54]:
# drive.mount('./gdrive/')
# dataset_folder = '/content/gdrive/MyDrive/Final'
# files = os.listdir(dataset_folder)
# for file in files:
#   if '.zip' in file:
#     file_path = os.path.join(dataset_folder, file)
#     with ZipFile(file_path, 'r') as f:
#       f.extractall()

## Get data and labels zip

In [55]:
protein_folder = './protein'
protein_list = os.listdir(protein_folder)

ligand_folder = './ligand'
ligand_list = os.listdir(ligand_folder)

label_folder = './label'
label_list = os.listdir(label_folder)

In [56]:
# protein_list

In [57]:
totalSize = len(ligand_list)
totalSize

16786

In [58]:
# permu = np.random.permutation(totalSize)
permu = np.random.RandomState(seed=69).permutation(totalSize)

In [59]:
train_num, validate_num, test_num = 0,0,0

In [60]:
iDataset_num = totalSize
ratio = (60,20,20)

In [61]:
train_num = int(iDataset_num * (ratio[0]/ (ratio[0]+ratio[1]+ratio[2])))
# val_num = int(iDataset_num * (ratio[1]/ (ratio[0]+ratio[1]+ratio[2])))
# test_num = int(iDataset_num * (ratio[2]/ (ratio[0]+ratio[1]+ratio[2])))

val_num = 100
test_num = 500
last_num = 2000

In [62]:
train_list_IDs = permu[:train_num]
val_list_IDs = permu[train_num:(train_num+val_num)]
test_list_IDs = permu[(train_num+val_num):(train_num+val_num+test_num)]
last_list_IDs = permu[(train_num+val_num+test_num):(train_num+val_num+test_num+last_num)]

In [63]:
train_list_IDs

array([ 1592, 14669,  5474, ...,  3035,  6871,  6499])

## Get features

In [64]:
def get_atom_features(atom, amino_acid, isprotein):
    ATOM_CODES = {}
    metals = ([3, 4, 11, 12, 13] + list(range(19, 32))
              + list(range(37, 51)) + list(range(55, 84))
              + list(range(87, 104)))
    atom_classes = [(5, 'B'), (6, 'C'), (7, 'N'), (8, 'O'), (15, 'P'), (16, 'S'), (34, 'Se'),
                    ([9, 17, 35, 53], 'halogen'), (metals, 'metal')]
    for code, (atomidx, name) in enumerate(atom_classes):
        if type(atomidx) is list:
            for a in atomidx:
                ATOM_CODES[a] = code
        else:
            ATOM_CODES[atomidx] = code
    try:
        classes = ATOM_CODES[atom.GetAtomicNum()]
    except:
        classes = 9

    possible_chirality_list = [
        Chem.rdchem.ChiralType.CHI_UNSPECIFIED,
        Chem.rdchem.ChiralType.CHI_TETRAHEDRAL_CW,
        Chem.rdchem.ChiralType.CHI_TETRAHEDRAL_CCW,
        Chem.rdchem.ChiralType.CHI_OTHER
    ]
    chirality = possible_chirality_list.index(atom.GetChiralTag())

    possible_formal_charge_list = [-5, -4, -3, -2, -1, 0, 1, 2, 3, 4, 5]
    try:
        charge = possible_formal_charge_list.index(atom.GetFormalCharge())
    except:
        charge = 11

    possible_hybridization_list = [
        Chem.rdchem.HybridizationType.S,
        Chem.rdchem.HybridizationType.SP,
        Chem.rdchem.HybridizationType.SP2,
        Chem.rdchem.HybridizationType.SP3,
        Chem.rdchem.HybridizationType.SP3D,
        Chem.rdchem.HybridizationType.SP3D2,
        Chem.rdchem.HybridizationType.UNSPECIFIED
    ]
    try:
        hyb = possible_hybridization_list.index(atom.GetHybridization())
    except:
        hyb = 6

    possible_numH_list = [0, 1, 2, 3, 4, 5, 6, 7, 8]
    try:
        numH = possible_numH_list.index(atom.GetTotalNumHs())
    except:
        numH = 9

    possible_implicit_valence_list = [0, 1, 2, 3, 4, 5, 6, 7]
    try:
        valence = possible_implicit_valence_list.index(atom.GetTotalValence())
    except:
        valence = 8

    possible_degree_list = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
    try:
        degree = possible_degree_list.index(atom.GetTotalDegree())
    except:
        degree = 11

    is_aromatic = [False, True]
    aromatic = is_aromatic.index(atom.GetIsAromatic())

    mass = atom.GetMass() / 100

    # idx = atom.GetIdx()
    # with open(protein_path, 'r+') as f:
    #     readlines = f.readlines()
    #     f.close()

    amino_acids = [
        'ALA', 'ARG', 'ASN', 'ASN', 'ASP', 'CYS', 'GLU', 'GLN', 'GLY', 'HIS', 'ILE', 'LEU', 'LYS', 'MET', 'PHE', 'PRO', 'SER', 'THR', 'TRP', 'TYR', 'VAL'
    ]
    if amino_acid in amino_acids:
      amino_acid = amino_acids.index(amino_acid)
    else:
      amino_acid = int(len(amino_acids) + 1)

    # amino_acid = amino_acids.index(amino_acid)
    # amino_acid = 0
    # for lines in readlines:
    #     if 'HETATM' in lines or 'ATOM' in lines:
    #         if idx == int(lines[6:11]):
    #             amino_acid = lines[17:20]
                # if amino_acid in amino_acids:
                #     amino_acid = amino_acids.index(amino_acid)
                # else:
                #     amino_acid = int(len(amino_acids) + 1)

    return [classes, chirality, charge, hyb, numH, valence, degree, aromatic, mass, amino_acid, isprotein]

## Get min coordinates

In [65]:
def get_min(compound_positions):
    minx,miny,minz = 999,999,999
    for pos in compound_positions:
        x, y, z = pos
        if x < minx:
            minx = x

        if y < miny:
            miny = y

        if z < minz:
            minz = z

    return (minx,miny,minz)

## Get grid

In [66]:
def adjust_grid(compound, compound_positions, protein_path, isprotein, grid, minx, miny, minz):
    """
    Adjusts the grid by adding features to specific grid positions based on the compound and protein information.

    Args:
        compound: The compound object.
        compound_positions: The positions of the compound atoms.
        protein_path: The path to the protein file.
        isprotein: A boolean indicating whether the compound is a protein.
        grid: The grid to be adjusted.
        minx: The minimum x-coordinate value.
        miny: The minimum y-coordinate value.
        minz: The minimum z-coordinate value.

    Returns:
        The adjusted grid.
    """
    atoms_aa = []
    with open(protein_path, 'r+') as f:
        readlines = f.readlines()
        f.close()

    for idx, lines in enumerate(readlines):
        if 'HETATM' in lines or 'ATOM' in lines:
            atoms_aa.append(lines[17:20])

    for idx, pos in enumerate(compound_positions):
        x, y, z = pos

        amino_acid = atoms_aa[idx]
        atom = compound.GetAtomWithIdx(int(idx))
        features = get_atom_features(atom, amino_acid, isprotein)
        features.extend(pos)

        if grid[round(x - minx), round(y - miny), round(z - minz)][0] == 0:
            grid[round(x - minx), round(y - miny), round(z - minz)] = features

        elif grid[round(x - minx)+1, round(y - miny), round(z - minz)][0] == 0:
            grid[round(x - minx)+1, round(y - miny), round(z - minz)] = features

        elif grid[round(x - minx), round(y - miny)+1, round(z - minz)][0] == 0:
            grid[round(x - minx), round(y - miny)+1, round(z - minz)] = features

        elif grid[round(x - minx), round(y - miny), round(z - minz)+1][0] == 0:
            grid[round(x - minx), round(y - miny), round(z - minz)+1] = features

        elif grid[round(x - minx)+1, round(y - miny)+1, round(z - minz)][0] == 0:
            grid[round(x - minx)+1, round(y - miny)+1, round(z - minz)] = features

        elif grid[round(x - minx), round(y - miny)+1, round(z - minz)+1][0] == 0:
            grid[round(x - minx), round(y - miny)+1, round(z - minz)+1] = features

        elif grid[round(x - minx)+1, round(y - miny), round(z - minz)+1][0] == 0:
            grid[round(x - minx)+1, round(y - miny), round(z - minz)+1] = features

        elif grid[round(x - minx)+1, round(y - miny)+1, round(z - minz)+1][0] == 0:
            grid[round(x - minx)+1, round(y - miny)+1, round(z - minz)+1] = features

    return grid

In [67]:
# def adjust_grid_ligand(compound, compound_positions, protein_path, isprotein, grid, minx, miny, minz):
#     for idx, pos in enumerate(compound_positions):
#         x,y,z = pos

#         atom = compound.GetAtomWithIdx(int(idx))
#         features = get_atom_features(atom, protein_path, isprotein)
#         features.extend(pos)

#         if grid[round(x - minx), round(y - miny), round(z - minz)][0] == 0:
#             grid[round(x - minx), round(y - miny), round(z - minz)] = features

#         elif grid[round(x - minx)+1, round(y - miny), round(z - minz)][0] == 0:
#             grid[round(x - minx)+1, round(y - miny), round(z - minz)] = features

#         elif grid[round(x - minx), round(y - miny)+1, round(z - minz)][0] == 0:
#             grid[round(x - minx), round(y - miny)+1, round(z - minz)] = features

#         elif grid[round(x - minx), round(y - miny), round(z - minz)+1][0] == 0:
#             grid[round(x - minx), round(y - miny), round(z - minz)+1] = features

#         elif grid[round(x - minx)+1, round(y - miny)+1, round(z - minz)][0] == 0:
#             grid[round(x - minx)+1, round(y - miny)+1, round(z - minz)] = features

#         elif grid[round(x - minx), round(y - miny)+1, round(z - minz)+1][0] == 0:
#             grid[round(x - minx), round(y - miny)+1, round(z - minz)+1] = features

#         elif grid[round(x - minx)+1, round(y - miny), round(z - minz)+1][0] == 0:
#             grid[round(x - minx)+1, round(y - miny), round(z - minz)+1] = features

#         elif grid[round(x - minx)+1, round(y - miny)+1, round(z - minz)+1][0] == 0:
#             grid[round(x - minx)+1, round(y - miny)+1, round(z - minz)+1] = features

#     return grid

In [68]:
def set_grid(protein_path, isprotein= 1):
    compound = Chem.MolFromPDBFile(protein_path, False, False, 1)
    compound_conf = compound.GetConformer()
    compound_positions = compound_conf.GetPositions()

    atom = compound.GetAtomWithIdx(int(1))
    features = get_atom_features(atom, '', isprotein)

    minx, miny, minz = get_min(compound_positions)
    # Generate empty grid with size 52x52x52x(features+3)
    grid=np.zeros((52,52,52,len(features)+3))

    adjusted_grid = adjust_grid(compound, compound_positions, protein_path, isprotein, grid, minx, miny, minz)

    return adjusted_grid, minx, miny, minz

## Add ligand

In [69]:
def add_ligand(ligand_path, grid, minx, miny, minz, isprotein = 0):
    ligand = Chem.MolFromPDBFile(ligand_path, False, False, 1)
    ligand_conf = ligand.GetConformer()
    ligand_positions = ligand_conf.GetPositions()

    grid = adjust_grid(ligand, ligand_positions, ligand_path, isprotein, grid, minx, miny, minz)

    return grid

## Create Protein_Grid_list

In [70]:
def create_grid_list(protein_folder):
    protein_list = os.listdir(protein_folder)
    grids = []
    for protein in protein_list:
        protein_name = protein.split('.')[0]
        protein_path = os.path.join(protein_folder, protein)

        grid, minx, miny, minz = set_grid(protein_path)
        grids.append((protein_name, grid, (minx, miny, minz)))
    return grids


## Get grid list

In [71]:
grids = create_grid_list(protein_folder)

In [72]:
grid = grids[0][1]
minx, miny, minz = grids[0][2]
name = grids[0][0]

In [73]:
new_grid = add_ligand('./ligand/3qzq-8v_model13.pdb', grid, minx, miny, minz)
new_grid

array([[[[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]],

        [[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]],

        [[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]],

        ...,

        [[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
    

In [74]:
# # for grid in grids:
count = 0
for x in new_grid:
    for y in x:
      for features in y:
        if features[0] != 0:
          # print(features)
          count+=1
print(count)

5514


## Get batch data

In [75]:
# /content/ligand/3qzq-10b_model1.pdb

In [76]:
def get_data_batch(dataset_idx, protein_folder, ligand_folder, ligand_list, label_folder, label_list, batch_size, index):
  """
  Retrieves a batch of data from the dataset.

  Args:
  - dataset_idx (list): List of indices representing the dataset.
  - protein_folder (str): Path to the folder containing protein files.
  - ligand_folder (str): Path to the folder containing ligand files.
  - ligand_list (list): List of ligand file names.
  - label_folder (str): Path to the folder containing label files.
  - label_list (list): List of label file names.
  - batch_size (int): Number of samples per batch.
  - index (int): Index of the batch to retrieve.

  Returns:
    tuple: A tuple containing the following arrays:
      - gridList (numpy.ndarray): Array of grid data.
      - baList (numpy.ndarray): Array of binding affinity values.
      - statList (numpy.ndarray): Array of hit/no-hit labels.
  """
  baList = []
  statList = []
  gridList = []

  batch_list = [value for idx, value in enumerate(dataset_idx) if idx >= index * batch_size and idx < (index+1)*batch_size ]
  
  for i in batch_list:
    complexFile = ligand_list[i]
    from_protein = complexFile.split('-')[0]
    complex_name = complexFile.split('.')[0]
    protein_path = os.path.join(protein_folder, from_protein + '.pdb')
    complexFile_path = os.path.join(ligand_folder, complexFile)
    grid, minx, miny, minz = set_grid(protein_path)
    grid = add_ligand(complexFile_path, grid, minx, miny, minz)
    gridList.append(grid)
    
    protein = [value for value in label_list if from_protein in value][0]
    label_file_path = os.path.join(label_folder, protein)

    df = pd.read_csv(label_file_path)
    # For each records in the dataframe,
    listidx = df.index[df['file.pdb'] == complex_name].tolist()[0]
    ba = df['BA'][listidx]
    stat = df['Hit/No_hit'][listidx]
    if stat == 'hit':
      stat = 1
    else:
      stat = 0
    baList.append(ba)
    statList.append(stat)
  
  gridList = np.array(gridList)
  baList = np.array(baList)
  statList = np.array(statList)

  return gridList, baList, statList


In [77]:
gridList, baList, statList = get_data_batch(train_list_IDs, protein_folder, ligand_folder, ligand_list, label_folder, label_list, 32, 0)

In [78]:
# Check if the grid is correct
# for grid in gridList:
#   count = 0
#   for x in grid:
#     for y in x:
#       for features in y:
#         if features[0] != 0:
#           # print(features)
#           count+=1
#   print(count)

In [79]:
# os.path.exists(path)
# tp = keras.metrics.TruePositives

## Get metric

In [80]:
def get_metrics(y_label, y_pred, ytype):

  if ytype == 0: # Regression
    print("++++++++++++++++++++++++++Regression++++++++++++++++++++++++++")
    PearsonR, _ = pearsonr(y_label, y_pred)
    print('Pearson Correlation Coefficient: ' + str(PearsonR))
    MSE = MeanAbsoluteError()
    MSE.update_state(y_label, y_pred)
    MSE = MSE.result().numpy()
    print('Mean Absolute Error: ' + str(MSE))
    # RMSE = math.sqrt(MSE)
    # print('Root Mean Absolute Error: ' + str(RMSE))

    return PearsonR, MSE

  if ytype == 1: # Classification
    print("+++++++++++++++++++++++Classification+++++++++++++++++++++++")
    # auc = keras.metrics.AUC()
    tp = keras.metrics.TruePositives(thresholds= 0.9)
    tn = keras.metrics.TrueNegatives(thresholds= 0.9)
    fp = keras.metrics.FalsePositives(thresholds= 0.9)
    fn = keras.metrics.FalseNegatives(thresholds= 0.9)
      
    # auc.update_state(y_label, y_pred)
    tp.update_state(y_label, y_pred)
    tn.update_state(y_label, y_pred)
    fp.update_state(y_label, y_pred)
    fn.update_state(y_label, y_pred)
    
    # auc = auc.result().numpy()
    tp_result = tp.result().numpy()
    tn_result = tn.result().numpy()
    fp_result = fp.result().numpy()
    fn_result = fn.result().numpy()
    print("TP Result:", tp_result)
    print("FP Result:", fp_result)
    precision = tp_result/ (tp_result+fp_result) # PPV
    print('Precision: ' + str(precision))

    # recall = Recall()
    # recall.update_state(y_label, y_pred)
    # recall = recall.result().numpy()
    recall = tp_result/(tp_result+fn_result) # Recall - TPR
    print('Recall: ' + str(recall))

    specificity = tn_result/(tn_result+fp_result)
    print('Specificity: ' + str(specificity))

    NPV = tn_result/(tn_result+fn_result)
    print('NPV: ' + str(NPV))

    # accuracy = Accuracy()
    # accuracy.update_state(y_label, y_pred)
    # accuracy = accuracy.result().numpy()
    # print('AUC: ' + str(auc))

    # f1_score = 2 * (precision * recall) / (precision + recall)
    # print('F1_Score: ' + str(f1_score))

    MCC = (tp_result*tn_result - fp_result*fn_result)/ math.sqrt((tp_result+fp_result)*(tp_result+fn_result)*(tn_result+fp_result)*(tn_result+fn_result)  ) # Phi coefficient
    print("Phi coefficient:" + str(MCC))

    return precision, recall, specificity, NPV, MCC

In [81]:
# def get_metrics_test(y_label, y_pred, ytype):

#   if ytype == 0: # Regression
#     print("++++++++++++++++++++++++++Regression++++++++++++++++++++++++++")
#     PearsonR, _ = pearsonr(y_label, y_pred)
#     print('Pearson Correlation Coefficient: ' + str(PearsonR))
#     MSE = MeanAbsoluteError()
#     MSE.update_state(y_label, y_pred)
#     MSE = MSE.result().numpy()
#     # print('Mean Absolute Error: ' + str(MSE))
#     RMSE = math.sqrt(MSE)
#     print('Root Mean Absolute Error: ' + str(RMSE))

#     return PearsonR, MSE, RMSE

#   if ytype == 1: # Classification
#     print("+++++++++++++++++++++++Classification+++++++++++++++++++++++")
#     auc = keras.metrics.AUC()
#     tp = keras.metrics.TruePositives(thresholds= 0.6)
#     tn = keras.metrics.TrueNegatives(thresholds= 0.6)
#     fp = keras.metrics.FalsePositives(thresholds= 0.6)
#     fn = keras.metrics.FalseNegatives(thresholds= 0.6)

#     auc.update_state(y_label, y_pred)
#     tp.update_state(y_label, y_pred)
#     tn.update_state(y_label, y_pred)
#     fp.update_state(y_label, y_pred)
#     fn.update_state(y_label, y_pred)

#     auc = auc.result().numpy()
#     tp = tp.result().numpy()
#     tn = tn.result().numpy()
#     fp = fp.result().numpy()
#     fn = fn.result().numpy()

#     print('True Positive: ' + str(tp))
#     print('True Negative: ' + str(tn))
#     print('False Positive: ' + str(fp))
#     print('False Negative: ' + str(fn))

#     # print("")
#     # precision = Precision()
#     # precision.update_state(y_label, y_pred)
#     # precision = precision.result().numpy()
#     precision = tp/ (tp+fp) # PPV
#     # print('Precision: ' + str(precision))

#     # recall = Recall()
#     # recall.update_state(y_label, y_pred)
#     # recall = recall.result().numpy()
#     recall = tp/(tp+fn) # Recall - TPR
#     # print('Recall: ' + str(recall))

#     # accuracy = Accuracy()
#     # accuracy.update_state(y_label, y_pred)
#     # accuracy = accuracy.result().numpy()
#     print('AUC: ' + str(auc))

#     f1_score = 2 * (precision * recall) / (precision + recall)
#     # print('F1_Score: ' + str(f1_score))

#     MCC = (tp*tn - fp*fn)/ math.sqrt(   (tp+fp)*(tp+fn)*(tn+fp)*(tn+fn)  ) # Phi coefficient
#     print("Phi coefficient:" + str(MCC))

#     return precision, recall, auc, f1_score, MCC

In [82]:
# csv_path = '/content/gdrive/MyDrive/Final/CSV/resultCSV.csv'

In [83]:
# csvfile = open(csv_path, 'w')

## Get Validate

In [84]:
import csv

In [85]:
def model_val_dataset(val_dataset_idx, protein_folder, label_folder, label_list, batch_size, save_path):
  dataset_len = len(val_dataset_idx)
  runs = dataset_len // batch_size
  last_batch = dataset_len - batch_size*runs
  model = keras.models.load_model(save_path+'.keras')
  # csv_path = 'resultCSV.csv'
  # csv_path = os.path.join(best_path, csv_path)
  # csvfile = open(csv_path, 'w')
  # fields = ['Model', 'Prediction C', 'Label C', 'Prediction R', 'Prediction R']
  # writer = csv.writer(f)
  # writer = csv.writer(csvfile)
  # writer.writerow(fields)
  # PearsonR_list, MCC_list, RSME_list = 0,0,0
  ba_Actual, stat_Actual, ba_Pred, stat_Pred = [], [], [], []
  print("----------------------- Start ValDataset ------------------------")
  for i in range(int(runs+1)):

    print("Get dataset on batch "+str(i+1))
    # If it is not the last batch
    if i != runs+1:
      gridList, baList, statList = get_data_batch(val_dataset_idx, protein_folder, ligand_folder, ligand_list, label_folder, label_list, batch_size, i)
    else:
      gridList, baList, statList = get_data_batch(val_dataset_idx, protein_folder, ligand_folder, ligand_list, label_folder, label_list, last_batch, i)

    print("----------------------- Predict ValDataset ------------------------")
    result = model.predict(gridList, verbose=2)
    gridList = []
    pred_reg, pred_class = result

    baList = [value for value in baList.tolist()]
    statList = [value for value in statList.tolist()]


    pred_reg = [value[0] for value in pred_reg.tolist()]
    pred_class = [value[0] for value in pred_class.tolist()]

    if ba_Actual == []:
      ba_Actual = baList
      stat_Actual = statList
      ba_Pred = pred_reg
      stat_Pred = pred_class
    else:
      ba_Actual.extend(baList)
      stat_Actual.extend(statList)
      ba_Pred.extend(pred_reg)
      stat_Pred.extend(pred_class)

    baList, statList = [], []
  print(ba_Actual, ba_Pred)
  PearsonR, MSE = get_metrics(ba_Actual, ba_Pred, 0)
  print("-------------------------------------------------------------")
  precision, recall, specificity, NPV, MCC = get_metrics(stat_Actual, stat_Pred, 1)
  # for idx, trueIndex in enumerate(val_dataset_idx):
  #   row = [str(trueIndex), str(stat_Actual[idx]), str(stat_Pred[idx]), str(ba_Actual[idx]), str(ba_Pred[idx])]
  #   print(row)
  #   writer.writerow(row)
  # csvfile.close()
  return PearsonR, MSE, precision, recall, specificity, NPV, MCC, ba_Actual, ba_Pred, stat_Actual, stat_Pred

## Get Train

In [86]:
def model_train_dataset(model: keras.Model, train_dataset_idx, val_dataset_idx, protein_folder, label_folder, label_list, batch_size, epochs, save_path, best_path):
  dataset_len = len(train_dataset_idx)
  runs = dataset_len // batch_size
  last_batch = dataset_len - batch_size*runs
  cur = 1
  # PearsonR, MSE, RMSE, precision, recall, auc, f1_score, MCC = 0,0,0,0,0,0,0,0
  # checkPearsonR, checkMCC, checkRMSE = 0,0,999
  log_txt = "log.txt"
  log_path = os.path.join(save_path, log_txt)
  readline = ''
  if os.path.exists(log_path):
    log_file = open(log_path,"r+")
    readline = log_file.readline()
    log_file.close()
  else:
    with open(log_path, 'w+') as f:
      f.write('0/'+str(runs))
      f.close()

  if readline == '' or int(readline.split('/')[0]) > runs or int(readline.split('/')[0]) == 0:
    cur = 1
    model.save(save_path + '.keras')
  else:
    cur = int(readline.split('/')[0])

  check = 0

  print("----------------------- Start TrainDataset ------------------------")
  for i in range(int(cur-1),int(runs+1)):
    print("=======================Batch "+ str(i+1)+"==============================")
    model = keras.models.load_model(save_path + '.keras')
    print("Get dataset")
    # get_data_batch(dataset_idx, protein_folder, ligand_folder, ligand_list, label_folder, label_list, batch_size, index)
    # get_data_batch(train_list_IDs, protein_folder, ligand_folder, ligand_list, label_folder, label_list, 32, 0)
    if i != runs+1:
      gridList, baList, statList = get_data_batch(train_dataset_idx, protein_folder, ligand_folder, ligand_list, label_folder, label_list, batch_size, i)
    else:
      gridList, baList, statList = get_data_batch(train_dataset_idx, protein_folder, ligand_folder, ligand_list, label_folder, label_list, last_batch, i)
    
    if len(gridList) == 0 or len(baList) == 0 or len(statList) == 0:
      raise ValueError(f'Empty List: gridList: {len(gridList)}, baList: {len(baList)}, statList: {len(statList)}')
    print("----------------------- Train TrainDataset ------------------------")
    model.fit(gridList, [baList, statList], epochs= epochs,verbose=0)
    gridList, baList, statList = [], [], []
    # PearsonR, MSE, RMSE, precision, recall, auc, f1_score, MCC = model_val_dataset(val_dataset_idx, protein_folder, label_folder, label_list, batch_size, epochs, save_path, best_path)
    print("Save")
    model.save(save_path+'.keras')
    log_file = open(log_path,"r+")
    readline = log_file.write(str(i)+'/'+str(runs))
    log_file.close()
    # if PearsonR > checkPearsonR and MCC > checkMCC and RMSE < checkRMSE:
    #   model.save(best_path)
    #   checkPearsonR = PearsonR
    #   checkMCC = MCC
    #   checkRMSE = RMSE
    if check == 0 and batch_size*i >= 2000:
      check +=1
      model.save(best_path + '.keras')
  return model

## Get Model

In [87]:
# def combine_embedding(drop_rate, input_shape= (60,60,60,14)):
#   inp = Input(shape= input_shape, name='Input_Complexes')

#   # Classification

#   ## Check there are atoms
#   x1 = Conv3D(64, (3, 3, 3), padding='same', bias_initializer='zeros', kernel_initializer='glorot_uniform')(inp)
#   x1 = BatchNormalization()(x1)
#   x1 = Activation('relu')(x1)
#   # x1 = MaxPooling3D(padding='same')(x1)
#   x1 = MaxPooling3D(strides= (2, 2, 2), padding='same')(x1)


#   x1 = Conv3D(128, (3, 3, 3), padding='same', bias_initializer='zeros', kernel_initializer='glorot_uniform')(x1)
#   x1 = BatchNormalization()(x1)
#   x1 = Activation('relu')(x1)
#   x1 = AveragePooling3D(padding='same')(x1)

#   x1 = Conv3D(256, (3, 3, 3), padding='same', bias_initializer='zeros', kernel_initializer='glorot_uniform')(x1)
#   x1 = Activation('relu')(x1)
#   x1 = BatchNormalization()(x1)
#   x1 = AveragePooling3D(padding='same')(x1)

#   # x1 = Conv3D(128, (3, 3, 3), padding='same', bias_initializer='zeros', kernel_initializer='glorot_uniform')(x1)
#   # x1 = BatchNormalization()(x1)
#   # x1 = Activation('relu')(x1)
#   # # x1 = AveragePooling3D(padding='same')(x1)
#   # x1 = AveragePooling3D(padding='same')(x1)

#   x1 = Conv3D(256, (3, 3, 3), padding='same', bias_initializer='zeros', kernel_initializer='glorot_uniform')(x1)
#   x1 = BatchNormalization()(x1)
#   x1 = Activation('relu')(x1)
#   x1 = AveragePooling3D(padding='same')(x1)

#   # Global Pooling
#   x2 = GlobalAveragePooling3D()(x1)

#   x2 = Dense(500, kernel_regularizer='l2', bias_initializer='zeros')(x2)
#   x2 = Dropout(rate=drop_rate)(x2)

#   # Flattening
#   x1 = Flatten()(x1)

#   x1 = Dense(4096, kernel_regularizer='l2', bias_initializer='zeros')(x1)
#   x1 = Dropout(rate=drop_rate)(x1)

#   x1 = Dense(4096, kernel_regularizer='l2', bias_initializer='zeros')(x1)
#   x1 = Dropout(rate=drop_rate)(x1)

#   x1 = Dense(1000, kernel_regularizer='l2', bias_initializer='zeros')(x1)
#   x1 = Dropout(rate=drop_rate)(x1)

#   # Regression Output
#   d1 = Dense(1, activation='linear')(x1)
#   # Classification Output
#   d2 = Dense(1, activation='sigmoid')(x2)

#   return Model(inputs=[inp], outputs=[d1,d2], name='Embedding')

In [88]:
def combine_embedding(drop_rate, input_shape= (52,52,52,11)):
  inp = Input(shape= input_shape, name='Input_Complexes')

  ## Check there are atoms
  x1 = Conv3D(filters= 8, kernel_size=(1,1,1), padding='same', bias_initializer='zeros', kernel_initializer='glorot_uniform')(inp)
  x1 = BatchNormalization()(x1)
  x1 = Activation('relu')(x1)

  x1 = Conv3D(8, kernel_size=(3,3,3))(x1)
  x1 = BatchNormalization()(x1)
  x1 = Activation('relu')(x1)

  x1 = Conv3D(32,kernel_size=(3,3,3),padding='same')(x1)
  x1 = BatchNormalization()(x1)
  x1 = Activation('relu')(x1)
  x1 = MaxPooling3D(pool_size=2)(x1)

  x1 = Conv3D(64,kernel_size=(1,1,1),padding='same')(x1)
  x1 = BatchNormalization()(x1)
  x1 = Activation('relu')(x1)

  x1 = Conv3D(64,kernel_size=(3,3,3),padding='same')(x1)
  x1 = BatchNormalization()(x1)
  x1 = Activation('relu')(x1)

  x1 = MaxPooling3D(pool_size=2)(x1)

  x1 = Conv3D(128,kernel_size=(1,1,1),padding='same')(x1)
  x1 = BatchNormalization()(x1)
  x1 = Activation('relu')(x1)

  x1 = Conv3D(128,kernel_size=(3,3,3),padding='same')(x1)
  x1 = BatchNormalization()(x1)
  x1 = Activation('relu')(x1)
  x1 = MaxPooling3D(pool_size=2)(x1)

  x1 = Conv3D(256,kernel_size=(1,1,1),padding='same')(x1)
  x1 = BatchNormalization()(x1)
  x1 = Activation('relu')(x1)

  x1 = Conv3D(256,kernel_size=(3,3,3),padding='same')(x1)
  x1 = BatchNormalization()(x1)
  x1 = Activation('relu')(x1)
  x1 = MaxPooling3D(pool_size=2)(x1)

    # Global Pooling
  x2 = GlobalAveragePooling3D()(x1)

  x2 = Dense(256)(x2)
  x2 = BatchNormalization()(x2)
  x2 = Activation('relu')(x2)
  x2 = Dropout(0.5)(x2)

  # Flattening
  x1 = Flatten()(x1)

  x1 = Dense(256)(x1)
  x1 = BatchNormalization()(x1)
  x1 = Activation('relu')(x1)
  x1 = Dropout(0.5)(x1)

  # Regression Output
  d1 = Dense(1,kernel_regularizer=keras.regularizers.l2(0.01))(x1)
  # Classification Output
  d2 = Dense(1, activation='sigmoid')(x2)

  return Model(inputs=[inp], outputs=[d1,d2], name='Embedding')

## Create model

In [89]:
fake_grid, x,y,z = set_grid('./protein/3qzq.pdb')

In [90]:
np.shape(fake_grid)

(52, 52, 52, 14)

In [91]:
shape = np.shape(fake_grid)
shape

(52, 52, 52, 14)

In [92]:
model = combine_embedding(0.5, shape)
model.summary()

Model: "Embedding"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ Input_Complexes     │ (None, 52, 52,    │          0 │ -                 │
│ (InputLayer)        │ 52, 14)           │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv3d_9 (Conv3D)   │ (None, 52, 52,    │        120 │ Input_Complexes[… │
│                     │ 52, 8)            │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 52, 52,    │         32 │ conv3d_9[0][0]    │
│ (BatchNormalizatio… │ 52, 8)            │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_11       │ (None, 52, 52,    │          0 │ batch_normalizat… │
│ (Activation)        │ 52, 8)            │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv3d_10 (Conv3D)  │ (None, 50, 50,    │      1,736 │ activation_11[0]… │
│                     │ 50, 8)            │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 50, 50,    │         32 │ conv3d_10[0][0]   │
│ (BatchNormalizatio… │ 50, 8)            │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_12       │ (None, 50, 50,    │          0 │ batch_normalizat… │
│ (Activation)        │ 50, 8)            │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv3d_11 (Conv3D)  │ (None, 50, 50,    │      6,944 │ activation_12[0]… │
│                     │ 50, 32)           │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 50, 50,    │        128 │ conv3d_11[0][0]   │
│ (BatchNormalizatio… │ 50, 32)           │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_13       │ (None, 50, 50,    │          0 │ batch_normalizat… │
│ (Activation)        │ 50, 32)           │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling3d_4     │ (None, 25, 25,    │          0 │ activation_13[0]… │
│ (MaxPooling3D)      │ 25, 32)           │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv3d_12 (Conv3D)  │ (None, 25, 25,    │      2,112 │ max_pooling3d_4[… │
│                     │ 25, 64)           │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 25, 25,    │        256 │ conv3d_12[0][0]   │
│ (BatchNormalizatio… │ 25, 64)           │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_14       │ (None, 25, 25,    │          0 │ batch_normalizat… │
│ (Activation)        │ 25, 64)           │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv3d_13 (Conv3D)  │ (None, 25, 25,    │    110,656 │ activation_14[0]… │
│                     │ 25, 64)           │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 25, 25,    │        256 │ conv3d_13[0][0]   │
│ (BatchNormalizatio… │ 25, 64)           │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_15       │ (None, 25, 25,    │          0 │ batch_normalizat

 Total params: 4,216,994 (16.09 MB)

 Trainable params: 4,214,082 (16.08 MB)

 Non-trainable params: 2,912 (11.38 KB)

In [93]:
optimizer=Adam(learning_rate=1e-4)

loss=['mean_squared_error', "binary_crossentropy"]
model.compile(optimizer= optimizer,
              loss= loss, run_eagerly=True)

## Save paths

In [94]:
# save_path = '/content/gdrive/MyDrive/Final/Model'
# best_path = '/content/gdrive/MyDrive/Final/Best'

In [95]:
save_path = './SFCNN/Model1'
best_path = './SFCNN/Best1'

## Set hyperparameters

In [47]:
batch_size = 32
epochs = 150

## Train

In [ ]:
# trained_model = model_train_dataset(model, train_list_IDs, val_list_IDs, protein_folder, label_folder, label_list, batch_size, epochs, save_path, best_path)

----------------------- Start TrainDataset ------------------------
=======================Batch 96==============================
Get dataset
----------------------- Train TrainDataset ------------------------


2024-04-05 09:04:51.279111: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:465] Loaded cuDNN version 8907


Save
=======================Batch 97==============================
Get dataset
----------------------- Train TrainDataset ------------------------
Save
=======================Batch 98==============================
Get dataset
----------------------- Train TrainDataset ------------------------
Save
=======================Batch 99==============================
Get dataset
----------------------- Train TrainDataset ------------------------
Save
=======================Batch 100==============================
Get dataset
----------------------- Train TrainDataset ------------------------
Save
=======================Batch 101==============================
Get dataset
----------------------- Train TrainDataset ------------------------
Save
=======================Batch 102==============================
Get dataset
----------------------- Train TrainDataset ------------------------
Save
=======================Batch 103==============================
Get dataset
----------------------- Train Train

# New test

In [ ]:
model = keras.models.load_model(save_path+'.keras')

In [ ]:
model.summary()

In [ ]:
# PearsonR, MSE, precision, recall, specificity, NPV, MCC = model_val_dataset(test_list_IDs, protein_folder, label_folder, label_list, batch_size, epochs, save_path, best_path)

In [ ]:
# !pip install mayavi

In [109]:
test1_list_IDs = permu[12000:12100]

In [ ]:
# len(test1_list_IDs)

In [98]:
PearsonR, MSE, precision, recall, specificity, NPV, MCC, ba_Actual, ba_Pred, stat_Actual, stat_Pred = model_val_dataset(val_list_IDs, protein_folder, label_folder, label_list, batch_size, save_path)

----------------------- Start ValDataset ------------------------
Get dataset on batch 1
----------------------- Predict ValDataset ------------------------
1/1 - 0s - 270ms/step
Get dataset on batch 2
----------------------- Predict ValDataset ------------------------
1/1 - 0s - 272ms/step
Get dataset on batch 3
----------------------- Predict ValDataset ------------------------
1/1 - 0s - 273ms/step
Get dataset on batch 4
----------------------- Predict ValDataset ------------------------
1/1 - 0s - 84ms/step
[-6.2, -6.5, -5.9, -6.7, -7.2, -6.4, -5.9, -6.3, -6.7, -6.0, -6.4, -7.1, -6.2, -7.5, -6.9, -7.3, -6.9, -7.3, -6.3, -6.8, -7.1, -6.2, -7.0, -6.8, -6.7, -7.6, -7.5, -6.3, -6.7, -6.3, -7.9, -7.4, -6.4, -5.8, -7.1, -6.5, -6.9, -6.7, -7.1, -6.3, -6.7, -6.6, -5.9, -7.1, -6.3, -6.7, -6.5, -7.2, -5.8, -8.1, -7.7, -6.4, -7.4, -7.0, -6.5, -6.1, -7.0, -6.4, -7.1, -6.4, -7.0, -7.0, -6.2, -6.6, -6.3, -6.1, -6.6, -6.7, -6.1, -7.0, -7.2, -6.6, -7.6, -6.6, -6.2, -7.7, -6.4, -7.2, -6.8, -6.9, -7

In [100]:
PearsonR, MSE, precision, recall, specificity, NPV, MCC, ba_Actual, ba_Pred, stat_Actual, stat_Pred = model_val_dataset(test_list_IDs, protein_folder, label_folder, label_list, batch_size, save_path)

----------------------- Start ValDataset ------------------------
Get dataset on batch 1
----------------------- Predict ValDataset ------------------------
1/1 - 0s - 271ms/step
Get dataset on batch 2
----------------------- Predict ValDataset ------------------------
1/1 - 0s - 271ms/step
Get dataset on batch 3
----------------------- Predict ValDataset ------------------------
1/1 - 0s - 270ms/step
Get dataset on batch 4
----------------------- Predict ValDataset ------------------------
1/1 - 0s - 276ms/step
Get dataset on batch 5
----------------------- Predict ValDataset ------------------------
1/1 - 0s - 273ms/step
Get dataset on batch 6
----------------------- Predict ValDataset ------------------------
1/1 - 0s - 266ms/step
Get dataset on batch 7
----------------------- Predict ValDataset ------------------------
1/1 - 0s - 272ms/step
Get dataset on batch 8
----------------------- Predict ValDataset ------------------------
1/1 - 0s - 271ms/step
Get dataset on batch 9
-------

In [ ]:
# len(stat_Pred)

In [101]:
fields = ['Model', 'Hit_Label', 'Hit_Prediction', 'BindingAffinity_Label', 'BindingAffinity_Prediction']

In [102]:
rows = []

In [103]:
for idx, value in enumerate(val_list_IDs):
  row = []
  row.append(ligand_list[value])
  row.append(stat_Actual[idx])
  row.append(stat_Pred[idx])
  row.append(ba_Actual[idx])
  row.append(ba_Pred[idx])
  # row = (ligand_list[value], stat_Actual[idx], stat_Pred[idx], ba_Actual[idx], ba_Pred[idx])
  # print(type(row))
  rows.append(row)
  # if rows == []:
  #   rows = row
  # else:
  #   rows.append(row)

In [104]:
rows

[['5gso-FIP_model97.pdb', 1, 0.31875425577163696, -6.5, -6.283574104309082],
 ['5c1u-CR_model9.pdb', 1, 0.2707501947879791, -7.2, -7.395552635192871],
 ['3sjo-HF_model80.pdb', 0, 0.197122722864151, -7.3, -6.558995246887207],
 ['5gso-GC376_model90.pdb', 0, 0.02720002643764019, -5.6, -6.271882057189941],
 ['5c1u-8v_model69.pdb', 0, 0.197122722864151, -7.5, -6.558995246887207],
 ['3sjk-NK19k_model27.pdb', 0, 0.02720002643764019, -5.6, -6.271882057189941],
 ['3qzr-F_model10.pdb', 1, 0.517372190952301, -7.2, -7.23667049407959],
 ['3qzr-FIP_model89.pdb', 0, 0.20664556324481964, -6.3, -6.736719131469727],
 ['7dnc-GC376_model48.pdb', 1, 0.2707501947879791, -8.1, -7.395552635192871],
 ['3sjo-CR_model74.pdb', 1, 0.02720002643764019, -6.2, -6.271882057189941],
 ['5gsw-NK19k_model98.pdb', 0, 0.8828364014625549, -6.5, -7.663809776306152],
 ['7dnc-AG_model60.pdb', 1, 0.9999401569366455, -7.2, -7.158212661743164],
 ['3sjo-AG_model33.pdb', 0, 0.20664556324481964, -6.5, -6.736719131469727],
 ['5c1u-8v_

In [105]:
len(rows)

100

In [106]:
csv_path = './Final/CSV/resultSFCNN.csv'

In [108]:
with open(csv_path, 'w') as csvfile:
    # creating a csv writer object
    csvwriter = csv.writer(csvfile)

    # writing the fields
    csvwriter.writerow(fields)

    # writing the data rows
    csvwriter.writerows(rows)

In [110]:
datatset_path = './Final/CSV/dataset.csv'

In [111]:
fields = ['Model', 'Protein', 'Ligand']

In [112]:
rows = []

In [113]:
count = 0

In [114]:
for idx, value in enumerate(test1_list_IDs):
  row = []
  row.append(idx+1)
  ligand = ligand_list[value]
  pro = ligand.split('-')[0]
  protein = [value for value in protein_list if pro in value][0]
  protein_name = protein.split('.')[0]
  ligand_name = ligand.split('.')[0]
  row.append(protein_name)
  row.append(ligand_name)
  count +=1

  rows.append(row)
  # if rows == []:
  #   rows = row
  # else:
  #   rows.append(row)

In [132]:
with open(datatset_path, 'w+') as csvfile:
    # creating a csv writer object
    csvwriter = csv.writer(csvfile)

    # writing the fields
    csvwriter.writerow(fields)

    # writing the data rows
    csvwriter.writerows(rows)

In [116]:
# rows

# Draw

In [117]:
try:
  import py3Dmol
except:
  %pip install py3Dmol
  import py3Dmol

Note: you may need to restart the kernel to use updated packages.


In [118]:
14748

14748

In [119]:
ligand_14748 = ligand_list[14748]
ligand_14748

'5c1u-HF_model29.pdb'

In [120]:
# ligand_14748_path = os.path.join(ligand_folder, ligand_14748)
hitLead_Ligand = './ligand/5c1u-CR_model22.pdb'

In [121]:
hitLead_Protein = './protein/5c1u.pdb'

In [122]:
# ligand_14748_path = os.path.join(ligand_folder, ligand_14748)
hit_Ligand = './ligand/5c1u-GC376_model97.pdb'

In [123]:
hit_Protein = './protein/3qzr.pdb'

In [124]:
no_Ligand = './ligand/5c1u-FOPMC_model71.pdb'

In [125]:
no_Protein = './protein/3qzr.pdb'

In [126]:
# view = py3Dmol.view()
# view.addModel(open(protein_5gso_path, 'r').read(),'pdb')
# view.setBackgroundColor('white')
# # view.setStyle({'chain':'A'}, {'cartoon': {'color':'purple'}})
# view.addStyle({'resn':'UH7'}, {'stick': {'colorscheme':'yellowCarbon'}})
# view.addStyle({'within':{'distance':'5', 'sel':{'resn':'UH7'}}}, {'stick': {}})
# # view.addSurface(py3Dmol.VDW, {'opacity':0.8, 'color':'blue'}, \
# #   {'not':{'or':[{'resn':'UH7'}, {'resn':'DMS'}]}})

# view.addModel(open(ligand_16545_path, 'r').read(),'pdb')
# view.addStyle({'resn':'UH7'}, {'stick': {'colorscheme':'yellowCarbon'}})
# view.addStyle({'within':{'distance':'5', 'sel':{'resn':'UH7'}}}, {'stick': {}})
# # view.addSurface(py3Dmol.VDW, {'opacity':0.9, 'color':'yellow'}, \
# #   {'not':{'or':[{'resn':'UH7'}, {'resn':'DMS'}]}})

# view.zoomTo()
# view.show()

In [127]:
view = py3Dmol.view()
view.removeAllModels()
view.setViewStyle({'style':'outline','color':'black','width':0.1})

view.addModel(open(hitLead_Protein,'r').read(),format='pdb')
Prot=view.getModel()
Prot.setStyle({'cartoon':{'arrows':True, 'tubes':True, 'style':'oval', 'color':'white'}})
view.addSurface(py3Dmol.VDW,{'opacity':0.8,'color':'white'})


view.addModel(open(hitLead_Ligand,'r').read(),format='mol2')
ref_m1 = view.getModel()
ref_m1.setStyle({},{'stick':{'colorscheme':'redCarbon','radius':0.2}})

view.addModel(open(hit_Ligand,'r').read(),format='mol2')
ref_m2 = view.getModel()
ref_m2.setStyle({},{'stick':{'colorscheme':'blueCarbon','radius':0.2}})

view.addModel(open(no_Ligand,'r').read(),format='mol2')
ref_m3 = view.getModel()
ref_m3.setStyle({},{'stick':{'colorscheme':'yellowCarbon','radius':0.2}})

# ref_m.setStyle({},{'cartoon':{'arrows':True, 'tubes':True, 'style':'oval', 'color':'red'}})
# ref_m.setStyle({'cartoon':{'arrows':True, 'tubes':True, 'style':'oval', 'color':'red'}})


# results=Chem.SDMolSupplier('1AZ8_lig_vina_out.sdf')

# p=Chem.MolToMolBlock(results[0],False)

# print('Reference: Magenta | Vina Pose: Cyan')
# print ('Pose: {} | Score: {}'.format(results[0].GetProp('Pose'),results[0].GetProp('Score')))

# view.addModel(p,'mol')
# x = view.getModel()
# x.setStyle({},{'stick':{'colorscheme':'cyanCarbon','radius':0.2}})

view.zoomTo()
view.show()

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [128]:
view = py3Dmol.view()
view.removeAllModels()
view.setViewStyle({'style':'outline','color':'black','width':0.1})

# view.addModel(open(hitLead_Protein,'r').read(),format='pdb')
# Prot=view.getModel()
# Prot.setStyle({'cartoon':{'arrows':True, 'tubes':True, 'style':'oval', 'color':'white'}})
# view.addSurface(py3Dmol.VDW,{'opacity':0.8,'color':'white'})


view.addModel(open(hitLead_Ligand,'r').read(),format='mol2')
ref_m = view.getModel()
ref_m.setStyle({},{'stick':{'colorscheme':'magentaCarbon','radius':0.2}})
# ref_m.setStyle({},{'cartoon':{'arrows':True, 'tubes':True, 'style':'oval', 'color':'red'}})
# ref_m.setStyle({'cartoon':{'arrows':True, 'tubes':True, 'style':'oval', 'color':'red'}})


# results=Chem.SDMolSupplier('1AZ8_lig_vina_out.sdf')

# p=Chem.MolToMolBlock(results[0],False)

# print('Reference: Magenta | Vina Pose: Cyan')
# print ('Pose: {} | Score: {}'.format(results[0].GetProp('Pose'),results[0].GetProp('Score')))

# view.addModel(p,'mol')
# x = view.getModel()
# x.setStyle({},{'stick':{'colorscheme':'cyanCarbon','radius':0.2}})

view.zoomTo()
view.show()

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [129]:
view = py3Dmol.view()
view.removeAllModels()
view.setViewStyle({'style':'outline','color':'black','width':0.1})

view.addModel(open(hitLead_Protein,'r').read(),format='pdb')
Prot=view.getModel()
Prot.setStyle({'cartoon':{'arrows':True, 'tubes':True, 'style':'oval', 'color':'white'}})
view.addSurface(py3Dmol.VDW,{'opacity':0.8,'color':'white'})


view.addModel(open(hitLead_Ligand,'r').read(),format='mol2')
ref_m = view.getModel()
ref_m.setStyle({},{'stick':{'colorscheme':'magentaCarbon','radius':0.2}})
# ref_m.setStyle({},{'cartoon':{'arrows':True, 'tubes':True, 'style':'oval', 'color':'red'}})
# ref_m.setStyle({'cartoon':{'arrows':True, 'tubes':True, 'style':'oval', 'color':'red'}})


# results=Chem.SDMolSupplier('1AZ8_lig_vina_out.sdf')

# p=Chem.MolToMolBlock(results[0],False)

# print('Reference: Magenta | Vina Pose: Cyan')
# print ('Pose: {} | Score: {}'.format(results[0].GetProp('Pose'),results[0].GetProp('Score')))

# view.addModel(p,'mol')
# x = view.getModel()
# x.setStyle({},{'stick':{'colorscheme':'cyanCarbon','radius':0.2}})

view.zoomTo()
view.show()

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [130]:
view = py3Dmol.view()
view.removeAllModels()
view.setViewStyle({'style':'outline','color':'black','width':0.1})

view.addModel(open(hit_Protein,'r').read(),format='pdb')
Prot=view.getModel()
Prot.setStyle({'cartoon':{'arrows':True, 'tubes':True, 'style':'oval', 'color':'white'}})
view.addSurface(py3Dmol.VDW,{'opacity':0.8,'color':'white'})


view.addModel(open(hit_Ligand,'r').read(),format='mol2')
ref_m = view.getModel()
ref_m.setStyle({},{'stick':{'colorscheme':'magentaCarbon','radius':0.2}})
# ref_m.setStyle({},{'cartoon':{'arrows':True, 'tubes':True, 'style':'oval', 'color':'red'}})
# ref_m.setStyle({'cartoon':{'arrows':True, 'tubes':True, 'style':'oval', 'color':'red'}})


# results=Chem.SDMolSupplier('1AZ8_lig_vina_out.sdf')

# p=Chem.MolToMolBlock(results[0],False)

# print('Reference: Magenta | Vina Pose: Cyan')
# print ('Pose: {} | Score: {}'.format(results[0].GetProp('Pose'),results[0].GetProp('Score')))

# view.addModel(p,'mol')
# x = view.getModel()
# x.setStyle({},{'stick':{'colorscheme':'cyanCarbon','radius':0.2}})

view.zoomTo()
view.show()

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [131]:
view = py3Dmol.view()
view.removeAllModels()
view.setViewStyle({'style':'outline','color':'black','width':0.1})

view.addModel(open(no_Protein,'r').read(),format='pdb')
Prot=view.getModel()
Prot.setStyle({'cartoon':{'arrows':True, 'tubes':True, 'style':'oval', 'color':'white'}})
view.addSurface(py3Dmol.VDW,{'opacity':0.8,'color':'white'})


view.addModel(open(no_Ligand,'r').read(),format='mol2')
ref_m = view.getModel()
ref_m.setStyle({},{'stick':{'colorscheme':'magentaCarbon','radius':0.2}})
# ref_m.setStyle({},{'cartoon':{'arrows':True, 'tubes':True, 'style':'oval', 'color':'red'}})
# ref_m.setStyle({'cartoon':{'arrows':True, 'tubes':True, 'style':'oval', 'color':'red'}})


# results=Chem.SDMolSupplier('1AZ8_lig_vina_out.sdf')

# p=Chem.MolToMolBlock(results[0],False)

# print('Reference: Magenta | Vina Pose: Cyan')
# print ('Pose: {} | Score: {}'.format(results[0].GetProp('Pose'),results[0].GetProp('Score')))

# view.addModel(p,'mol')
# x = view.getModel()
# x.setStyle({},{'stick':{'colorscheme':'cyanCarbon','radius':0.2}})

view.zoomTo()
view.show()

3Dmol.js failed to load for some reason. Please check your browser console for error messages.